# CI/CD on GCP Project 

# Continuous Development and Continuous Delivery on Google Cloud

### Plan -> Code -> Build -> Test -> Release -> Deploy -> Operate -> Monitor

## Objective

* To build a CI/CD Pipeline on GCP using Google CLoud Services

### Planning Stage

#### Architecture Diagram

### Coding Stage

#### Directory

In [ ]:
!mkdir app

In [ ]:
cd app

#### Database

##### Postgres Docker

In [ ]:
# Without volume/data connected
# !docker run --name postgres-sql -e POSTGRES_USER=matt -e POSTGRES_PASSWORD=password  -d postgres

In [ ]:
# With volume/data connected
!docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=matt \
    -e POSTGRES_PASSWORD=password \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres

In [ ]:
!docker ps

In [ ]:
!ls -a

##### pgAdmin

In [ ]:
!docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

In [ ]:
!docker ps

##### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:8501,tcp:9000 --source-ranges=0.0.0.0/0

#### Application

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit psycopg2-binary google-cloud-aiplatform -q
COPY . /app
EXPOSE 9000
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0", "--server.port", "9000"]

### Building, Testing, and Releasing Stages

In [ ]:
# Enable Artifact Registry, Cloud Build, and Cloud Run

#### Create an Artifact Registry

In [ ]:
import os
# Environment Variables
REGION="us-central1"
APP_ARTIFACT_NAME="app"
APP_NAME="app"
APP_VERSION="latest"
os.environ['REGION']=REGION
os.environ["APP_ARTIFACT_NAME"]=APP_ARTIFACT_NAME
os.environ["APP_NAME"]=APP_NAME
os.environ["APP_VERSION"]=APP_VERSION

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create $APP_ARTIFACT_NAME \
    --repository-format=docker \
    --location=$REGION \
    --description="Docker repository"

In [ ]:
%%sh
# Verify
gcloud artifacts repositories list

#### Build an image using Dockerfile and submit to Artifact Registry

In [ ]:
# Check if the path is correct
!pwd

In [ ]:
cd app

In [ ]:
!gcloud builds submit \
    --region=$REGION \
    --tag $REGION-docker.pkg.dev/$(gcloud config get-value project)/$APP_NAME/$APP_NAME:$APP_VERSION

### Deploying Stage

In [ ]:
# Deploy using the console

In [ ]:
# Deploy using cli

### Operating Stage

### Monitoring Stage

## Conclusion